In [40]:
import src.ettj_getter.scrapper as scrapper
import pandas as pd

In [ ]:
# It can take a long time... It is recomended to break it in smaller pieces...
start_date = "01/01/2000"
end_date = "30/06/2025"

result_scrapper = scrapper.baixar_ajuste_bmf_multiplos_dias(start_date, end_date)

result_scrapper = result_scrapper[result_scrapper["Mercadoria"] == "DI1 - DI de 1 dia"]

In [ ]:
result_scrapper.to_csv("./data/ETTJ/ettj_2000_2025.csv", index=False)

In [2]:
result_scrapper = pd.read_csv("./data/ETTJ/ettj_2000_2025.csv")
result_scrapper["Data Pregão"] = pd.to_datetime(result_scrapper["Data Pregão"])
result_scrapper

,Data Pregão,Mercadoria,Vct,Preço de Ajuste Anterior,Preço de Ajuste Atual,Variação,Valor do Ajuste por Contrato (R$)
0,2000-01-03,DI1 - DI de 1 dia,JAN0,100000.84,100000.00,-0.84,0.84
1,2000-01-03,DI1 - DI de 1 dia,FEV0,98554.86,98553.63,-1.23,1.23
2,2000-01-03,DI1 - DI de 1 dia,MAR0,97086.61,97090.83,4.22,4.22
3,2000-01-03,DI1 - DI de 1 dia,ABR0,95615.84,95605.00,-10.84,10.84
4,2000-01-03,DI1 - DI de 1 dia,MAI0,94274.01,94240.00,-34.01,34.01
...,...,...,...,...,...,...,...
198765,2025-06-30,DI1 - DI de 1 dia,F36,26435.23,27229.53,794.30,794.30
198766,2025-06-30,DI1 - DI de 1 dia,F37,23310.85,24079.95,769.10,769.10
198767,2025-06-30,DI1 - DI de 1 dia,F38,20677.28,21419.81,742.53,742.53
198768,2025-06-30,DI1 - DI de 1 dia,F39,18339.84,19052.56,712.72,712.72


#### Adjust Vct for the ones in wrong format

In [ ]:
def adjust_vct_for_old_pregoes(df):

    MONTH_CODES = {"F": 1,
               "G": 2,
               "H": 3,
               "J": 4,
               "K": 5,
               "M": 6,
               "N": 7,
               "Q": 8,
               "U": 9,
               "V": 10,
               "X": 11,
               "Z": 12}
    
    WRONG_MONTH_MAP = {"JAN": "F",
                    "FEV": "G",
                    "MAR": "H",
                    "ABR": "J",
                    "MAI": "K",
                    "JUN": "M",
                    "JUL": "N",
                    "AGO": "Q",
                    "SET": "U",
                    "OUT": "V",
                    "NOV": "X",
                    "DEZ": "Z"}

    WRONG_MONTH_LIST = list(WRONG_MONTH_MAP.keys())

    WRONG_YEAR_MAP = {"00'": {0: 2000, 1: 2001, 2: 2002, 3: 2003, 4: 2004, 5: 2005, 6: 2006, 7: 2007, 8: 2008, 9: 2009},
                  "10'": {0: 2010, 1: 2011, 2: 2012, 3: 2013, 4: 2014, 5: 2015, 6: 2016, 7: 2017, 8: 2018, 9: 2019},
                  "20'": {0: 2020, 1: 2021, 2: 2022, 3: 2023, 4: 2024, 5: 2025, 6: 2026, 7: 2027, 8: 2028, 9: 2029}}
    
    pregoes = df["Data Pregão"].unique().tolist()

    for preg in pregoes:
        df_preg = df[df["Data Pregão"] == preg]
        for index, row in df_preg.iterrows():
            WRONG_YEAR_MAP_copy = WRONG_YEAR_MAP.copy()

            ano_pregao = row["Data Pregão"].year
            zero_zero_dict = {key: ano for key, ano in WRONG_YEAR_MAP_copy["00'"].items() if ano >= ano_pregao}
            WRONG_YEAR_MAP_copy["00'"] = zero_zero_dict

            if row["Vct"][0:3] in WRONG_MONTH_LIST:
                right_month = WRONG_MONTH_MAP[row["Vct"][0:3]]
                code_year_vcto = int(row["Vct"][-1])

                if code_year_vcto in WRONG_YEAR_MAP_copy["00'"].keys():
                    right_year = WRONG_YEAR_MAP_copy["00'"][code_year_vcto]
                elif code_year_vcto in WRONG_YEAR_MAP_copy["10'"].keys():
                    right_year = WRONG_YEAR_MAP_copy["10'"][code_year_vcto]
                elif code_year_vcto in WRONG_YEAR_MAP_copy["20'"].keys():
                    right_year = WRONG_YEAR_MAP_copy["20'"][code_year_vcto]
                else:
                    print("Erro inesperado")
                    print(row)
                    break

                right_vct = f"{right_month}{str(right_year)[-2:]}"
                df_preg.at[index, "Vct"] = right_vct

        df.loc[df_preg.index, "Vct"] = df_preg["Vct"]
    
    return df


In [ ]:
result_scrapper_adjusted = adjust_vct_for_old_pregoes(result_scrapper)

In [ ]:
result_scrapper_adjusted.to_csv("./data/ETTJ/ettj_right_vct.csv", index=False)